In [1]:
from rockpool import TSEvent, TSContinuous
from rockpool.nn.modules import LIFTorch, LinearTorch
from rockpool.nn.combinators import Sequential
import rockpool.devices.xylo.syns61201 as x
from rockpool.transform import quantize_methods as q
import snntorch.spikegen as spikegen

Looking in indexes: https://gitlab.com/api/v4/projects/27423070/packages/pypi/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.7/140.7 MB 3.1 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: samna
    Found existing installation: samna 0.36.4.0
    Uninstalling samna-0.36.4.0:
      Successfully uninstalled samna-0.36.4.0


ModuleNotFoundError: No module named 'bitstruct'

In [ ]:
input_size = 1
hidden_size = 16
output_size = 1
dt = 1e-3

net = Sequential(
    LinearTorch((input_size, hidden_size), has_bias=False),
    LIFTorch(hidden_size, dt=dt),
    LinearTorch((hidden_size, output_size), has_bias=False),
    LIFTorch(output_size, dt=dt)
)
print(net)

TorchSequential  with shape (1, 1) {
    LinearTorch '0_LinearTorch' with shape (1, 16)
    LIFTorch '1_LIFTorch' with shape (16, 16)
    LinearTorch '2_LinearTorch' with shape (16, 1)
    LIFTorch '3_LIFTorch' with shape (1, 1)
}


In [ ]:
# - Call the Xylo mapper on the extracted computational graph
spec = x.mapper(net.as_graph(),  weight_dtype='float', threshold_dtype='float', dash_dtype='float')

# - Quantize the specification
spec.update(q.global_quantize(**spec))

# - Use rockpool.devices.xylo.config_from_specification to convert it to a hardware configuration
config, is_valid, msg = x.config_from_specification(**spec)
if not is_valid:
    print(msg)

ModuleNotFoundError: Missing the `samna` backend. `config_from_specification` objects, and others relying on `samna` are not available.

In [ ]:
# - Configure the simulator with the HW network config
modSim = x.XyloSim.from_config(config, dt=dt)
print(modSim)

In [ ]:
# Create the native data from glucose data
T = 1000
spikes_in = spikegen.rate(0.1, T)

In [ ]:
# - Evolve the input over the network, in simulation
out, _, r_d = modSim(spikes_in, record=True)

# - Show the internal state variables recorded
print(r_d.keys())

In [ ]:
# - Plot some internal state variables
plt.figure()
plt.imshow(r_d['Spikes'].T, aspect = 'auto', origin = 'lower')
plt.title('Hidden spikes')
plt.ylabel('Channel')

plt.figure()
TSContinuous.from_clocked(r_d['Isyn'], dt, name = 'Hidden synaptic currents').plot(stagger = 127);

plt.figure()
TSContinuous.from_clocked(r_d['Vmem'], dt, name = 'Hidden membrane potentials').plot(stagger = 127);